# Build Chains and Results 

In [1]:
data_file_name = 'EMS_DCMA_DD_23.08.graphml'
experiment = None
tasks = 'milestones' # 'milestones' for milestones only, not yet available
results = 'prt'# 'chains' for a list of chains only, not yet available
from calculate import *
if not experiment: experiment = data_file_name.replace('.graphml', '').replace('.', '_')
run_calculation_process(data_file_name, experiment, tasks, results)

Starting EC2 instance: i-0586e11281d4b02a2
EC2 instance "i-0586e11281d4b02a2" has been started
Data file uploaded to S3
Run attempt encountered error:
 []
Calculation finished
Compute instance stopped
Preparing results


ClientError: An error occurred (404) when calling the HeadObject operation: Not Found

# Query Results  

In [ ]:
import duckdb
con = duckdb.connect()
results_files = "'{e}/*.parquet'".format(e=experiment)
query = """SELECT * FROM {rf} """.format(rf=results_files)
query_df = con.execute(query).df()
#query_df.to_excel('query_df.xlsx')
display(query_df.head(20))

In [ ]:
len(query_df)